# **MODEL EVALUATION NOTEBOOK**

## Objectives

* Write here your notebook objective, for example, "Fetch data from Kaggle and save as raw data", or "engineer features for modelling"

## Inputs

* Write here which data or information you need to run the notebook 

## Outputs

* Write here which files, code or artefacts you generate by the end of the notebook 

## Additional Comments

* In case you have any additional comments that don't fit in the previous bullets, please state them here. 


---

# Change working directory

In [ ]:
import os
from pathlib import Path

# Resolve the project root
project_root = Path.cwd()
if project_root.name == "jupyter_notebooks":
    project_root = project_root.parent

# Import Libraries

In this section, all necessary standard libaries are imported to allow using their functions.

Import Libraries with necessary Settings

In [ ]:
# Standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import joblib
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Settings
%matplotlib inline
sns.set(style="whitegrid")

---

# Load Model Evaluation Results & Train and Test Sets

In this section, the model evaluation results are loaded to be able to access the prepared data.

In [ ]:
evaluation_path = project_root / "outputs" / "evaluation"

df_training = pd.read_csv(evaluation_path / "model_training_results.csv")
df_tuning = pd.read_csv(evaluation_path / "model_tuning_results.csv")
df_fe = pd.read_csv(evaluation_path / "best_model_fe_results.csv")
df_final = pd.read_csv(evaluation_path / "best_model_final_results.csv")


data_path = project_root / "outputs" / "data"

X_test = pd.read_csv(data_path / "X_test.csv")
X_test_scaled = pd.read_csv(data_path / "X_test_scaled.csv")
X_train = pd.read_csv(data_path / "X_train.csv")
X_train_resampled = pd.read_csv(data_path / "X_train_resampled.csv")
X_train_scaled = pd.read_csv(data_path / "X_train_scaled.csv")
y_test = pd.read_csv(data_path / "y_test.csv")
y_train = pd.read_csv(data_path / "y_train.csv")
y_train_resampled = pd.read_csv(data_path / "y_train_resampled.csv")

---

# Prepare Results for Visualisation

In this section, 

In [ ]:
# Add a column to indicate the stage
df_training['stage'] = 'Baseline'
df_tuning['stage'] = 'Tuning'
df_fe['stage'] = 'Feature Engineering'
df_final['stage'] = 'Final Test'

# Combine all into one
df_all = pd.concat([df_training, df_tuning, df_fe, df_final], ignore_index=True)

# Clean model names
df_all['Model'] = df_all['Model'].str.replace("_", " ").str.title()

* 

---

# Visualisations

In this section, 

Set general file path for saving plots

In [ ]:
figure_path = project_root / "outputs" / "eda" / "figures"

## Plot Metric Comparisons by Stage

In [ ]:
metrics = ['Accuracy', 'Balanced Accuracy', 'F1 Macro', 'F1 Weighted']

for metric in metrics:
    plt.figure(figsize=(10, 6))
    sns.barplot(data=df_all, x=metric, y='Model', hue='stage', palette='Set2')
    plt.title(f"{metric.replace('_', ' ').title()} by Model and Stage")
    plt.xlabel(metric.title())
    plt.ylabel("Model")
    plt.legend(title="Evaluation Stage")
    plt.tight_layout()
    plt.savefig(figure_path / f"models_{metric}_stage_comparison.png")
    plt.show()

* 

## Plot Confusion Matrix for Final Model

In [ ]:
# Load final model and test data
final_model_path = project_root / "outputs" / "ml_pipeline" / "default_best_model.pkl"
final_model = joblib.load(final_model_path)

# Predict
y_pred = final_model.predict(X_test_scaled)

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=final_model.classes_)

# Plot and save
plt.figure(figsize=(8, 6))
disp.plot(cmap="Blues", values_format='d')
plt.title("Confusion Matrix: Final Random Forest")
plt.savefig(figure_path / "final_model_confusion_matrix.png")
plt.show()

## Plot Feature Importance for Final Model

In [ ]:
# Get feature importances
feature_importances = final_model.feature_importances_
feature_names = X_train_resampled.columns  # match features to importance

# Create DataFrame
df_feat_imp = pd.DataFrame({
    'feature': feature_names,
    'importance': feature_importances
}).sort_values(by='importance', ascending=False)

# Plot top 15
plt.figure(figsize=(10, 6))
sns.barplot(data=df_feat_imp.head(15), x='importance', y='feature', palette='viridis')
plt.title("Top 15 Feature Importances - Final Random Forest")
plt.tight_layout()
plt.savefig(figure_path / "final_feature_importances.png")
plt.show()

---

# Conclusion and Next Steps

* 